In [ ]:
!pip install tensorflow
!pip install tensorflow-addons
!pip install roboflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 5.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=e3de4862227c76347aee598924a6fb73abbcd652a6c5f8fb7749f99a9824024b
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfull

In [ ]:

from roboflow import Roboflow
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Download the dataset
rf = Roboflow(api_key="bvgx6NFEHR0zZurL8c9w")
project = rf.workspace("northeastern-4sfxe").project("open-hole-detection-construction-safety")
dataset = project.version(3).download("retinanet")


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Open-hole-Detection---Construction-Safety-3 in retinanet:: 100%|██████████| 417/417 [00:00<00:00, 900.85it/s]


In [ ]:
# Path to the dataset directory
dataset_dir = "/content/Open-hole-Detection---Construction-Safety-3/"


In [ ]:

# Read the CSV annotation files
train_annotations = pd.read_csv(dataset_dir + "train/_annotations.csv")
valid_annotations = pd.read_csv(dataset_dir + "valid/_annotations.csv")

In [ ]:
# Path to the dataset directory
dataset_dir = "/content/Open-hole-Detection---Construction-Safety-3/"

# Read the CSV annotation files
train_annotations = pd.read_csv(dataset_dir + "train/_annotations.csv")
valid_annotations = pd.read_csv(dataset_dir + "valid/_annotations.csv")

# Load the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_dir + "train/",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=4,
    image_size=(640, 640),
    shuffle=True,
    seed=123,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False
)

# Load the validation dataset
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_dir + "valid/",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=4,
    image_size=(640, 640),
    shuffle=False,
    seed=123,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False
)

# Retrieve the class names
class_names = train_dataset.class_names

# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(640, 640, 3))

# Add custom layers for object detection
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(len(class_names), activation="sigmoid")(x)

# Create the RetinaNet model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(lr=0.001),
    loss=BinaryCrossentropy(),
    metrics=["accuracy"]
)

# Train the model
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=10
)

Found 0 files belonging to 0 classes.


ValueError: ignored

In [ ]:

# Load the dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'Open-hole-Detection---Construction-Safety-3/train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(640, 640),
    batch_size=4
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'Open-hole-Detection---Construction-Safety-3/valid',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(640, 640),
    batch_size=4
)

class_names = train_dataset.class_names



Found 0 files belonging to 0 classes.
Using 0 files for training.


ValueError: ignored

In [ ]:
# Preprocess the dataset
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))



In [ ]:
# Define the model
num_classes = len(class_names)
base_model = ResNet50(input_shape=(640, 640, 3), include_top=False, weights='imagenet')
base_model.trainable = False

input_tensor = tf.keras.Input(shape=(640, 640, 3))
x = normalization_layer(input_tensor)
x = base_model(x, training=False)
x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.Conv2D(num_classes * 4, (3, 3), activation='linear', padding='same')(x)
x = tf.keras.layers.Reshape((-1, 4))(x)

model = tf.keras.Model(inputs=input_tensor, outputs=x)

# Define the loss function
def smooth_l1_loss(y_true, y_pred):
    return tf.losses.Huber()(y_true, y_pred)

# Compile the model
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss=smooth_l1_loss)




In [ ]:
# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=10)


In [ ]:

# Save the trained model
model.save('retinanet_model.h5')